# Autocorrelation Matters: HAC Variance and Model Comparison

## Why Naive Standard Errors Are Wrong for Time Series

---

## 🚨 If You Know sklearn But Not HAC Variance, Read This First

**What you already know (from standard ML)**:
- `ttest_rel` or `ttest_ind` compare model performance
- Standard errors: `SE = σ / √n`
- p < 0.05 means "statistically significant"
- Errors are independent (that's what makes the math work)

**What's different with time series**:

In cross-sectional ML, residuals are independent. In time series, they're **correlated**.

**The problem**:
```python
# Standard ML: errors are independent
errors = [0.1, -0.2, 0.3, -0.1, 0.2]  # Each unrelated to others

# Time series: errors are correlated!
# If error at t is positive, error at t+1 is likely positive too
errors = [0.1, 0.12, 0.08, -0.1, -0.08]  # They cluster!
```

**Why this breaks standard tests**:

| Assumption | Standard ML | Time Series | Impact |
|------------|-------------|-------------|--------|
| Error independence | ✓ True | ✗ Violated | SE too small |
| Variance formula | σ²/n | HAC needed | p-values too small |
| Naive t-test | Valid | **Invalid** | False significance |

**Example**:
- Naive t-test: p = 0.02 → "Significant!" 
- Proper HAC test: p = 0.15 → "Not significant"
- You claimed a win that wasn't real.

---

**What you'll learn:**
1. Why forecast errors are autocorrelated (MA(h-1) structure for h-step forecasts)
2. How autocorrelation inflates naive variance estimates → spurious significance
3. How to use HAC (Heteroskedasticity and Autocorrelation Consistent) variance
4. How to properly compare models with the Diebold-Mariano test

**Prerequisites:** Notebooks 01-03

---

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

from temporalcv.cv import WalkForwardCV
from temporalcv.statistical_tests import dm_test, compute_hac_variance

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete.")

---

## Section 0: ACF and PACF Intuition

Before diving into HAC variance, let's build intuition for autocorrelation.

### What ACF Tells You

**ACF (Autocorrelation Function)**: Correlation between a series and its lagged values.

| ACF(lag) | Meaning | Implication |
|----------|---------|-------------|
| ACF(1) = 0.95 | y[t] and y[t-1] are 95% correlated | Very sticky series |
| ACF(1) = 0.50 | y[t] and y[t-1] are 50% correlated | Moderate memory |
| ACF(1) = 0.10 | y[t] and y[t-1] are 10% correlated | Nearly random |

### ACF Decay Patterns

```
Fast Decay (φ=0.3):           Slow Decay (φ=0.9):
ACF: [1.0, 0.3, 0.09, 0.03]   ACF: [1.0, 0.9, 0.81, 0.73]
     ▇▂░░                           ▇▇▇▆

→ Easy to beat persistence      → Hard to beat persistence
```

### What PACF Tells You

**PACF (Partial Autocorrelation Function)**: Correlation at lag k after removing effects of shorter lags.

| Pattern | Interpretation | Model |
|---------|---------------|-------|
| PACF drops after lag 1 | Direct dependence only on y[t-1] | AR(1) |
| PACF drops after lag p | Direct dependence on last p values | AR(p) |
| PACF decays slowly | Complex dependence structure | Consider differencing |

### Reading ACF/PACF Plots

```
                    ACF Plot                    PACF Plot
                ┌─────────────┐              ┌─────────────┐
Significance → ─ ─ ─ ─ ─ ─ ─ ─  (95% CI)    ─ ─ ─ ─ ─ ─ ─ ─
                █ █ ▆ ▅ ▄ ▃ ▂   (AR(1))       █ ▁ ▁ ▁ ▁ ▁ ▁
                █ █ █ █ █ █ █   (Random Walk) █ ▁ ▁ ▁ ▁ ▁ ▁
                └─────────────┘              └─────────────┘
                  lag →                        lag →
```

**Key insight**: If ACF decays slowly → high persistence → hard to beat persistence → need HAC variance for inference.

In [ ]:
# Visualize ACF decay for different persistence levels
def compute_acf_visual(series, max_lag=15):
    """Compute ACF for visualization."""
    n = len(series)
    mean_s = np.mean(series)
    var_s = np.var(series)
    
    acf = []
    for lag in range(max_lag + 1):
        if lag == 0:
            acf.append(1.0)
        else:
            cov = np.mean((series[lag:] - mean_s) * (series[:-lag] - mean_s))
            acf.append(cov / var_s)
    return np.array(acf)

# Generate AR(1) with different φ
np.random.seed(42)
n = 500

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

phi_values = [0.3, 0.7, 0.95]
labels = ['Low (φ=0.3)\nEasy to Beat', 'Moderate (φ=0.7)\nAchievable', 'High (φ=0.95)\nVery Hard']
colors = ['green', 'orange', 'red']

for ax, phi, label, color in zip(axes, phi_values, labels, colors):
    # Generate AR(1)
    y = np.zeros(n)
    y[0] = np.random.normal(0, 1 / np.sqrt(1 - phi**2))
    for t in range(1, n):
        y[t] = phi * y[t-1] + np.random.normal()
    
    # Compute ACF
    acf = compute_acf_visual(y, max_lag=15)
    
    # Plot
    lags = np.arange(len(acf))
    ax.bar(lags, acf, color=color, alpha=0.7, edgecolor='black')
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.axhline(y=1.96/np.sqrt(n), color='gray', linestyle='--', alpha=0.7)
    ax.axhline(y=-1.96/np.sqrt(n), color='gray', linestyle='--', alpha=0.7)
    
    ax.set_xlabel('Lag')
    ax.set_ylabel('ACF')
    ax.set_title(label, fontsize=11, fontweight='bold', color=color)
    ax.set_ylim(-0.3, 1.1)

plt.suptitle('ACF Decay Comparison: Persistence Difficulty', fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("★ Key Insight: Slow ACF decay = high persistence = hard to beat baseline")
print("  - Fast decay (φ=0.3): ACF drops to near-zero within 5 lags")
print("  - Slow decay (φ=0.95): ACF stays significant for 15+ lags")

In [ ]:
# Generate AR(1) process
def generate_ar1(n=500, phi=0.9, sigma=1.0, seed=42):
    """Generate AR(1) process."""
    rng = np.random.default_rng(seed)
    y = np.zeros(n)
    y[0] = rng.normal(0, sigma / np.sqrt(1 - phi**2))
    for t in range(1, n):
        y[t] = phi * y[t-1] + sigma * rng.normal()
    return y

def create_lag_features(series, n_lags=5):
    """Create lag features."""
    n = len(series)
    X = np.column_stack([
        np.concatenate([[np.nan]*lag, series[:-lag]]) 
        for lag in range(1, n_lags + 1)
    ])
    valid = ~np.isnan(X).any(axis=1)
    return X[valid], series[valid]

# Generate data
series = generate_ar1(n=600, phi=0.9, seed=42)
print(f"Generated AR(1) series with ACF(1) = {np.corrcoef(series[1:], series[:-1])[0,1]:.3f}")

---

## Section 1: The MA(h-1) Error Structure [T1]

### Why are h-step forecast errors autocorrelated?

For an AR(1) process:
$$y_{t+h} = \phi^h y_t + \sum_{j=0}^{h-1} \phi^j \epsilon_{t+h-j}$$

The **optimal h-step forecast** is:
$$\hat{y}_{t+h|t} = \phi^h y_t$$

The **forecast error** is:
$$e_{t+h|t} = y_{t+h} - \hat{y}_{t+h|t} = \sum_{j=0}^{h-1} \phi^j \epsilon_{t+h-j}$$

This is an **MA(h-1) process**! The errors are correlated up to lag h-1.

### The Intuition

- **h=1 forecasts**: Errors are uncorrelated (pure innovation)
- **h=2 forecasts**: Errors at t and t+1 share the shock at t+1
- **h=12 forecasts**: Errors overlap by 11 shocks → high correlation

In [ ]:
# Demonstrate error autocorrelation for different horizons
def compute_forecast_errors(series, horizon, n_lags=5):
    """Compute h-step forecast errors using AR model."""
    X, y = create_lag_features(series, n_lags=n_lags)
    
    # Split 80-20
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # For simplicity, use same model but interpret as h-step
    # (In practice, you'd use a proper h-step ahead target)
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    # Simulate h-step error structure by creating overlapping errors
    # This simulates what happens with multi-step forecasts
    errors = y_test - preds
    if horizon > 1:
        # Smooth errors to simulate MA(h-1) structure
        kernel = np.ones(horizon) / np.sqrt(horizon)
        errors = np.convolve(errors, kernel, mode='valid')
    
    return errors

def compute_acf(errors, max_lag=10):
    """Compute autocorrelation function."""
    n = len(errors)
    mean_e = np.mean(errors)
    var_e = np.var(errors)
    
    acf = []
    for lag in range(max_lag + 1):
        if lag == 0:
            acf.append(1.0)
        else:
            cov = np.mean((errors[lag:] - mean_e) * (errors[:-lag] - mean_e))
            acf.append(cov / var_e)
    return np.array(acf)

# Compute errors and ACF for different horizons
horizons = [1, 4, 8, 12]
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for ax, h in zip(axes.flat, horizons):
    errors = compute_forecast_errors(series, h)
    acf = compute_acf(errors, max_lag=15)
    
    lags = np.arange(len(acf))
    ax.bar(lags, acf, color='steelblue', alpha=0.7, edgecolor='black')
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.axhline(y=1.96/np.sqrt(len(errors)), color='red', linestyle='--', 
               label='95% CI', alpha=0.7)
    ax.axhline(y=-1.96/np.sqrt(len(errors)), color='red', linestyle='--', alpha=0.7)
    
    # Mark the expected cutoff at h-1
    if h > 1:
        ax.axvline(x=h-1, color='green', linestyle=':', linewidth=2, 
                   label=f'Expected cutoff (h-1={h-1})')
    
    ax.set_xlabel('Lag')
    ax.set_ylabel('ACF')
    ax.set_title(f'h={h} Forecast Error ACF', fontsize=12, fontweight='bold')
    ax.legend(loc='upper right', fontsize=8)
    ax.set_ylim(-0.5, 1.1)

plt.suptitle('[T1] h-Step Forecast Errors Have MA(h-1) Structure', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\nKey insight: h-step forecast errors are correlated up to lag h-1.")
print("This violates the i.i.d. assumption in naive standard errors.")

---

## Section 2: The Problem with Naive Variance

**Naive variance**: $\text{Var}(\bar{d}) = \frac{\text{Var}(d)}{n}$

This assumes errors are uncorrelated. When they're positively correlated:
- Naive variance is **too small**
- Standard errors are **too small**
- p-values are **too small**
- You claim significance when there is none

In [ ]:
# Demonstrate the variance underestimation
def simulate_variance_comparison(n=200, phi=0.7, n_simulations=1000, seed=42):
    """Simulate the difference between naive and true variance."""
    rng = np.random.default_rng(seed)
    
    means_naive = []
    means_true = []
    
    for i in range(n_simulations):
        # Generate AR(1) series (correlated errors)
        series = np.zeros(n)
        series[0] = rng.normal()
        for t in range(1, n):
            series[t] = phi * series[t-1] + rng.normal()
        
        means_true.append(np.mean(series))
    
    # True variance of sample mean
    true_var = np.var(means_true)
    
    # Naive estimate (assumes i.i.d.)
    # For AR(1): Var(series) = sigma^2 / (1 - phi^2)
    theoretical_var = 1 / (1 - phi**2)
    naive_var_of_mean = theoretical_var / n
    
    return {
        'true_var': true_var,
        'naive_var': naive_var_of_mean,
        'ratio': true_var / naive_var_of_mean
    }

print("NAIVE vs TRUE VARIANCE OF SAMPLE MEAN")
print("=" * 55)
print("\n(Simulating correlated series with different φ values)\n")

for phi in [0.0, 0.3, 0.7, 0.9]:
    result = simulate_variance_comparison(n=200, phi=phi)
    print(f"φ={phi}:")
    print(f"  Naive Var(mean): {result['naive_var']:.6f}")
    print(f"  True Var(mean):  {result['true_var']:.6f}")
    print(f"  Ratio (true/naive): {result['ratio']:.1f}x")
    if result['ratio'] > 1.5:
        print(f"  → Naive SE underestimates by ~{np.sqrt(result['ratio']):.1f}x")
    print()

In [ ]:
# Show the practical consequence: false significance
def compute_false_positive_rate(n=100, phi=0.7, true_diff=0, n_simulations=1000, seed=42):
    """Compute false positive rate using naive vs HAC variance."""
    rng = np.random.default_rng(seed)
    
    false_positives_naive = 0
    false_positives_hac = 0
    
    for i in range(n_simulations):
        # Generate correlated "loss differentials" (no true difference)
        d = np.zeros(n)
        d[0] = rng.normal()
        for t in range(1, n):
            d[t] = phi * d[t-1] + rng.normal()
        d = d + true_diff  # Add true difference (0 under null)
        
        mean_d = np.mean(d)
        
        # Naive test
        naive_se = np.std(d) / np.sqrt(n)
        naive_t = mean_d / naive_se
        naive_pvalue = 2 * (1 - stats.t.cdf(abs(naive_t), df=n-1))
        if naive_pvalue < 0.05:
            false_positives_naive += 1
        
        # HAC test
        hac_var = compute_hac_variance(d, bandwidth=int(phi * 10))
        if hac_var > 0:
            hac_se = np.sqrt(hac_var)
            hac_t = mean_d / hac_se
            hac_pvalue = 2 * (1 - stats.t.cdf(abs(hac_t), df=n-1))
            if hac_pvalue < 0.05:
                false_positives_hac += 1
    
    return {
        'naive_fpr': false_positives_naive / n_simulations,
        'hac_fpr': false_positives_hac / n_simulations
    }

print("FALSE POSITIVE RATES AT α=0.05")
print("=" * 55)
print("\n(Expected: 5% under the null hypothesis)\n")

for phi in [0.0, 0.3, 0.7, 0.9]:
    result = compute_false_positive_rate(phi=phi)
    print(f"φ={phi}:")
    print(f"  Naive FPR: {result['naive_fpr']*100:.1f}%", end='')
    if result['naive_fpr'] > 0.10:
        print(f"  ← INFLATED!")
    else:
        print()
    print(f"  HAC FPR:   {result['hac_fpr']*100:.1f}%")
    print()

---

## Section 3: HAC Variance Estimation [T1]

**HAC (Heteroskedasticity and Autocorrelation Consistent)** variance corrects for serial correlation.

The Newey-West estimator:

$$\hat{\text{Var}}_{\text{HAC}} = \frac{1}{n} \left[ \gamma_0 + 2 \sum_{j=1}^{m} w_j \gamma_j \right]$$

Where:
- $\gamma_j$ = autocovariance at lag j
- $w_j$ = Bartlett kernel weight: $1 - j/(m+1)$
- $m$ = bandwidth (typically h-1 for h-step forecasts)

### Reference [T1]
Newey & West (1987): "A Simple, Positive Semi-Definite, Heteroskedasticity and Autocorrelation Consistent Covariance Matrix"

In [ ]:
# Demonstrate compute_hac_variance
print("compute_hac_variance Demo")
print("=" * 55)

# Generate correlated series
n = 200
rng = np.random.default_rng(42)
d = np.zeros(n)
d[0] = rng.normal()
for t in range(1, n):
    d[t] = 0.7 * d[t-1] + rng.normal()

# Naive variance
naive_var = np.var(d) / n

# HAC variance with different bandwidths
print(f"\nSeries with φ=0.7, n={n}\n")
print(f"{'Bandwidth':<15} {'HAC Var':<15} {'Ratio to Naive'}")
print("-" * 45)
print(f"{'Naive (b=0)':<15} {naive_var:<15.6f} 1.0x")

for bandwidth in [1, 3, 5, 10, 'auto']:
    if bandwidth == 'auto':
        hac_var = compute_hac_variance(d)  # Uses automatic bandwidth
        label = 'Auto'
    else:
        hac_var = compute_hac_variance(d, bandwidth=bandwidth)
        label = f'b={bandwidth}'
    
    ratio = hac_var / naive_var if naive_var > 0 else float('inf')
    print(f"{label:<15} {hac_var:<15.6f} {ratio:.1f}x")

print("\n[T1] HAC variance is larger → accounts for correlation.")

In [ ]:
# Visualize the Bartlett kernel
def bartlett_kernel(j, bandwidth):
    """Bartlett kernel weight."""
    if abs(j) <= bandwidth:
        return 1.0 - abs(j) / (bandwidth + 1)
    return 0.0

fig, ax = plt.subplots(figsize=(10, 5))

bandwidths = [3, 5, 10]
colors = ['blue', 'green', 'red']

for bw, color in zip(bandwidths, colors):
    lags = np.arange(0, 15)
    weights = [bartlett_kernel(j, bw) for j in lags]
    ax.plot(lags, weights, 'o-', color=color, linewidth=2, markersize=8, 
            label=f'Bandwidth = {bw}', alpha=0.7)

ax.set_xlabel('Lag (j)', fontsize=12)
ax.set_ylabel('Kernel Weight', fontsize=12)
ax.set_title('Bartlett Kernel Weights for HAC Variance', fontsize=13, fontweight='bold')
ax.legend()
ax.set_ylim(-0.1, 1.1)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nThe kernel gives full weight to lag 0, then declines linearly to 0 at bandwidth+1.")
print("For h-step forecasts, use bandwidth = h-1 (errors are MA(h-1)).")

---

## Section 4: The Diebold-Mariano Test [T1]

To compare two forecasting models, use the **Diebold-Mariano (DM) test**.

Tests H₀: $E[d_t] = 0$ where $d_t = L(e_{1,t}) - L(e_{2,t})$ is the loss differential.

The DM statistic:

$$\text{DM} = \frac{\bar{d}}{\sqrt{\widehat{\text{Var}}_{\text{HAC}}(\bar{d})}}$$

**Key features:**
1. Uses HAC variance (handles autocorrelated errors)
2. Harvey et al. (1997) small-sample correction
3. Works for any loss function (squared, absolute)

### Reference [T1]
Diebold & Mariano (1995): "Comparing Predictive Accuracy"

In [ ]:
# Demonstrate dm_test
print("dm_test Demo")
print("=" * 55)

# Generate two sets of forecast errors
# Model 1: AR-based forecast (slightly better)
# Model 2: Persistence baseline

X, y = create_lag_features(series, n_lags=5)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Model 1: Ridge regression
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)
preds_model = model.predict(X_test)
errors_model = y_test - preds_model

# Model 2: Persistence
preds_persist = X_test[:, 0]  # y[t-1]
errors_persist = y_test - preds_persist

# Run DM test for different horizons
print(f"\nComparing Ridge model vs Persistence baseline\n")
print(f"Model MAE: {np.mean(np.abs(errors_model)):.4f}")
print(f"Persistence MAE: {np.mean(np.abs(errors_persist)):.4f}")

print(f"\n{'Horizon':<10} {'DM Stat':<12} {'p-value':<12} {'Conclusion'}")
print("-" * 50)

for h in [1, 4, 8, 12]:
    result = dm_test(
        errors_1=errors_model,
        errors_2=errors_persist,
        h=h,
        loss='absolute',
        alternative='less',  # H1: model is better (lower loss)
        harvey_correction=True
    )
    
    conclusion = 'Model better' if result.pvalue < 0.05 else 'Not significant'
    print(f"h={h:<8} {result.statistic:<12.3f} {result.pvalue:<12.4f} {conclusion}")

print("\n[T1] Note: As h increases, HAC bandwidth increases → more conservative test.")

In [ ]:
# Show the full result object
result = dm_test(
    errors_1=errors_model,
    errors_2=errors_persist,
    h=4,
    loss='absolute',
    alternative='less',
    harvey_correction=True
)

print("DMTestResult object (h=4)")
print("=" * 55)
print(f"\nstatistic:     {result.statistic:.4f}")
print(f"pvalue:        {result.pvalue:.4f}")
print(f"mean_diff:     {result.mean_loss_diff:.6f}")
print(f"horizon:       {result.h}")
print(f"n:             {result.n}")
print(f"loss:          {result.loss}")
print(f"alternative:   {result.alternative}")

---

## Section 5: Practical Implications

### When to Use HAC Variance?

**Always** when comparing time series forecasts, because:
1. h > 1 forecasts have MA(h-1) errors by construction
2. Even h = 1 forecasts can have autocorrelated errors if the model is misspecified
3. Naive tests are anti-conservative → too many false positives

### Bandwidth Selection

- **For h-step forecasts**: Use bandwidth = h - 1 (theoretical minimum)
- **Unknown h**: Use automatic selection (Andrews 1991 rule)
- **Conservative**: Use larger bandwidth if in doubt

In [ ]:
# Show the effect of bandwidth on DM test results
print("Bandwidth Sensitivity Analysis")
print("=" * 55)

print(f"\nDM test: Model vs Persistence (same data)\n")
print(f"{'Bandwidth':<12} {'DM Stat':<12} {'p-value':<12} {'SE(diff)'}")
print("-" * 48)

# Compute loss differentials
loss_model = np.abs(errors_model)
loss_persist = np.abs(errors_persist)
d = loss_model - loss_persist
mean_d = np.mean(d)

for bw in [0, 1, 3, 5, 10]:
    hac_var = compute_hac_variance(d, bandwidth=bw) if bw > 0 else np.var(d) / len(d)
    if hac_var > 0:
        se = np.sqrt(hac_var)
        dm_stat = mean_d / se
        pvalue = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
        print(f"{bw:<12} {dm_stat:<12.3f} {pvalue:<12.4f} {se:.6f}")
    else:
        print(f"{bw:<12} {'N/A':<12} {'N/A':<12} {'Negative var'}")

print("\nNote: Higher bandwidth → larger SE → more conservative test.")

In [ ]:
# Complete workflow: proper model comparison
def compare_models_properly(series, horizon=4):
    """
    Compare two models using proper HAC-based inference.
    """
    X, y = create_lag_features(series, n_lags=5)
    
    # Walk-forward CV with proper gap
    cv = WalkForwardCV(
        n_splits=5,
        extra_gap=horizon,
        window_type='expanding',
        test_size=50
    )
    
    all_errors_model = []
    all_errors_persist = []
    
    for train_idx, test_idx in cv.split(X):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        
        # Model predictions
        model = Ridge(alpha=1.0)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        
        # Persistence predictions
        persist_preds = X_test[:, 0]
        
        # Collect errors
        all_errors_model.extend(y_test - preds)
        all_errors_persist.extend(y_test - persist_preds)
    
    errors_model = np.array(all_errors_model)
    errors_persist = np.array(all_errors_persist)
    
    # DM test with proper HAC variance
    result = dm_test(
        errors_1=errors_model,
        errors_2=errors_persist,
        h=horizon,
        loss='absolute',
        alternative='less',
        harvey_correction=True
    )
    
    return {
        'model_mae': np.mean(np.abs(errors_model)),
        'persist_mae': np.mean(np.abs(errors_persist)),
        'dm_stat': result.statistic,
        'pvalue': result.pvalue,
        'significant': result.pvalue < 0.05
    }

print("COMPLETE MODEL COMPARISON WORKFLOW")
print("=" * 60)

for h in [1, 4, 8]:
    result = compare_models_properly(series, horizon=h)
    
    print(f"\nHorizon h={h}:")
    print(f"  Model MAE:     {result['model_mae']:.4f}")
    print(f"  Persist. MAE:  {result['persist_mae']:.4f}")
    print(f"  DM statistic:  {result['dm_stat']:.3f}")
    print(f"  p-value:       {result['pvalue']:.4f}")
    print(f"  Significant:   {'Yes' if result['significant'] else 'No'} (α=0.05)")

---

## Pitfall Section

### Pitfall 1: Using Naive t-test for Model Comparison

```python
# WRONG: Naive t-test ignores autocorrelation
from scipy.stats import ttest_rel
stat, pvalue = ttest_rel(errors_model, errors_persist)
# p-value is too small → false significance!

# RIGHT: DM test with HAC variance
result = dm_test(errors_model, errors_persist, h=horizon)
# p-value is correct
```

### Pitfall 2: Ignoring Horizon in HAC Bandwidth

```python
# WRONG: Default bandwidth for h=12 forecast
result = dm_test(e1, e2, h=1)  # Pretending it's 1-step

# RIGHT: Use correct horizon
result = dm_test(e1, e2, h=12)  # Bandwidth will be 11
```

### Pitfall 3: Claiming Significance Without Adjustment

```python
# WRONG: Report raw improvement percentage
improvement = (persist_mae - model_mae) / persist_mae * 100
print(f"Model is {improvement:.1f}% better!")  # No significance test!

# RIGHT: Report with statistical test
result = dm_test(errors_model, errors_persist, h=horizon)
if result.pvalue < 0.05:
    print(f"Model significantly better (DM p={result.pvalue:.3f})")
else:
    print(f"Improvement not significant (DM p={result.pvalue:.3f})")
```

In [ ]:
# Demonstrate the pitfalls
from scipy.stats import ttest_rel

print("Pitfall Demonstrations")
print("=" * 55)

# Use the errors from earlier
print("\nPitfall 1: Naive t-test vs DM test")
print("-" * 40)

# Naive t-test (WRONG)
loss_model = np.abs(errors_model)
loss_persist = np.abs(errors_persist)
stat_naive, pvalue_naive = ttest_rel(loss_model, loss_persist)
print(f"Naive t-test: stat={stat_naive:.3f}, p={pvalue_naive:.4f}")

# DM test (RIGHT)
result_dm = dm_test(errors_model, errors_persist, h=4, loss='absolute')
print(f"DM test (h=4): stat={result_dm.statistic:.3f}, p={result_dm.pvalue:.4f}")

print(f"\n→ Naive p-value: {pvalue_naive:.4f}")
print(f"→ DM p-value:    {result_dm.pvalue:.4f}")
print(f"\nThe naive test is anti-conservative (p-value too small).")

---

## Key Insights

### 1. h-Step Forecast Errors are MA(h-1) [T1]
Multi-step forecast errors are autocorrelated up to lag h-1 by construction.

### 2. Naive Variance Underestimates [T1]
Assuming i.i.d. errors produces standard errors that are too small → false significance.

### 3. HAC Variance Corrects the Problem [T1]
The Newey-West estimator accounts for autocorrelation using the Bartlett kernel.

### 4. DM Test for Model Comparison [T1]
Use dm_test() with the correct horizon to properly compare forecast accuracy.

### 5. Bandwidth = h - 1 [T1]
For h-step forecasts, use bandwidth = h - 1 to capture the MA(h-1) error structure.

---

## Next Steps

- **05_shuffled_target_gate.ipynb**: Definitive leakage detection with permutation tests
- **09_statistical_tests_dm_pt.ipynb**: Deep dive into DM and Pesaran-Timmermann tests
- **08_validation_workflow.ipynb**: Complete HALT/WARN/PASS pipeline

---

*"Never use naive standard errors for time series. Always use HAC variance."*